<a href="https://colab.research.google.com/github/sebavassou/dio_bootcamp/blob/main/transferlearning0002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

import os

#if using Theano with GPU
#os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [12]:
!tar -xzf 101_ObjectCategories.tar.gz
!rm 101_ObjectCategories.tar.gz
!ls

101_ObjectCategories  sample_data


In [13]:
root = '101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

['101_ObjectCategories/ferry', '101_ObjectCategories/hawksbill', '101_ObjectCategories/ketch', '101_ObjectCategories/snoopy', '101_ObjectCategories/cougar_face', '101_ObjectCategories/octopus', '101_ObjectCategories/windsor_chair', '101_ObjectCategories/wheelchair', '101_ObjectCategories/mayfly', '101_ObjectCategories/grand_piano', '101_ObjectCategories/flamingo', '101_ObjectCategories/chair', '101_ObjectCategories/dalmatian', '101_ObjectCategories/tick', '101_ObjectCategories/menorah', '101_ObjectCategories/sea_horse', '101_ObjectCategories/trilobite', '101_ObjectCategories/bonsai', '101_ObjectCategories/revolver', '101_ObjectCategories/cellphone', '101_ObjectCategories/gerenuk', '101_ObjectCategories/pigeon', '101_ObjectCategories/ibis', '101_ObjectCategories/accordion', '101_ObjectCategories/wrench', '101_ObjectCategories/helicopter', '101_ObjectCategories/Leopards', '101_ObjectCategories/umbrella', '101_ObjectCategories/panda', '101_ObjectCategories/chandelier', '101_ObjectCategori


Esta função é útil para pré-processar os dados em uma imagem e um vetor de entrada.

In [14]:
# função auxiliar para carregar a imagem e retorná-la e inserir o vetor
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [ ]:
random.shuffle(data)